## Image example

In [27]:
from PIL import Image
import numpy as np
import torch
# 000000093965# Load and preprocess the image
image = Image.open("/home/bibhabasum/projects/IIIT/data/coco/val2017/000000397133.jpg")
image = image.resize((640, 640))  # Resize to model input size
image = np.asarray(image).astype(np.float32) / 255.0  # Normalize to [0, 1]
image = torch.tensor(image).permute(2, 0, 1).unsqueeze(0)  # Convert to tensor and add batch dimension
# image = torch.concat([image, image],axis=0)

In [28]:
from model import DetectionModel

model = DetectionModel(nc = 80)
    # print(model)
weights = torch.load('/home/bibhabasum/projects/IIIT/ultralytics/model_state_dict.pth',map_location="cpu") # Load weights
model.load_state_dict(weights, strict=True)  # Load model weights

model.eval()

output = model(image) 

/tmp/ipykernel_3434970/2002377975.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load('/home/bibhabasum/projects/IIIT/ultralytics/model_state_dict.pth',

In [29]:
output.shape

torch.Size([1, 300, 6])

In [33]:
from utils import non_max_suppression

final = non_max_suppression(output,0.5,0.7)

In [34]:
final

[tensor([[3.8518e+02, 1.0337e+02, 4.9894e+02, 5.2086e+02, 8.8513e-01, 0.0000e+00],
         [1.2550e-01, 3.9502e+02, 6.1564e+01, 4.6465e+02, 7.6010e-01, 0.0000e+00],
         [5.7164e+01, 4.3292e+02, 1.3615e+02, 4.9117e+02, 7.5265e-01, 4.5000e+01],
         [1.5434e+02, 2.5250e+02, 1.8292e+02, 2.7496e+02, 6.3679e-01, 4.5000e+01],
         [4.8683e+02, 3.0134e+02, 6.2021e+02, 5.1356e+02, 6.1026e-01, 6.9000e+01],
         [3.1458e-01, 2.9124e+02, 1.9438e+02, 4.4756e+02, 5.2711e-01, 6.9000e+01]],
        grad_fn=<IndexBackward0>)]

In [35]:
import cv2

from classes_dict import classes_dict
from IPython.display import display, Image as IPImage

# Convert the image tensor to a numpy array and scale it back to [0, 255]
image_np = (image.squeeze().permute(1, 2, 0).numpy() * 255).astype(np.uint8)

# Draw bounding boxes
for detection in final[0]:  # Iterate over detections
    x1, y1, x2, y2, conf, cls = detection.tolist()
    class_name = classes_dict.get(int(cls), "Unknown")
    label = f"{class_name} {conf:.2f}"
    
    # Draw rectangle
    cv2.rectangle(image_np, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
    
    # Put label
    cv2.putText(image_np, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Display the image (if running locally)
# display(IPImage("temp_output.jpg"))
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# Save the image with bounding boxes
cv2.imwrite("output_with_bboxes.jpg", image_np)

True